In [4]:
import numpy as np
import pandas as pd
from scipy.signal import resample
import tqdm as tqdm

In [3]:
def resample_ecg(ecg_data,origHz,targetHz):
    resample_factor=targetHz/origHz
    resampled_ecg=resample(ecg_data,int(len(ecg_data)*resample_factor))
    return resampled_ecg

In [12]:
array=pd.read_csv("A00001_ecg.csv")
targetHz=360
CurrentHz=300
new_ecg=resample_ecg(array,CurrentHz,targetHz)

In [18]:
df=pd.DataFrame(new_ecg)
df.to_csv("A00001_resamp_ecg.csv",index=False)

In [24]:
def save_resample(Num):
    array=pd.read_csv(f"A{Num:05d}_ecg.csv")
    targetHz=360
    CurrentHz=300
    new_ecg=resample_ecg(array,CurrentHz,targetHz)
    df=pd.DataFrame(new_ecg)
    df.to_csv(f"A{Num:05d}_resamp_ecg.csv",index=False)

In [28]:
import tqdm as tqdm

for i in tqdm.tqdm(range(8528)):
    save_resample(i+1)

100%|██████████| 8528/8528 [02:43<00:00, 52.09it/s]


In [2]:
#Now we have the resampled data, we can re-arrange the data for the number of seconds for the model
Seconds=3
size_recording=Seconds*360 #get the number of records the seconds corresponds to
#Now define a function that splits the record into the second chunks
def split_record(Num):
    array=np.array(pd.read_csv(f"A{Num:05d}_resamp_ecg.csv"))
    startNum=0#Keeps track of the start of the record
    length=array.shape[0]
    new_array=array[startNum:(startNum+size_recording)].T
    startNum+=size_recording
    while startNum<length:
        add_array=array[startNum:(startNum+size_recording)].T
        if add_array.shape[1]<size_recording:
            #If the length of the recording is shorter than the timeframe, pad it at the end
            add_array=np.pad(add_array,pad_width=((0,0),(0,size_recording-add_array.shape[1])))
        new_array=np.vstack([new_array,add_array])
        startNum+=size_recording
    #Now we can save our numpy array back in a new file:
    data=pd.DataFrame(new_array)
    data.to_csv(f"A{Num:05d}_{Seconds}secs_ecg.csv",index=False)

In [5]:
for i in tqdm.tqdm(range(8528)):
    split_record(i+1)

100%|██████████| 8528/8528 [02:20<00:00, 60.70it/s]
